In [2]:
%pip install elasticsearch

  Obtaining dependency information for elasticsearch from https://files.pythonhosted.org/packages/bb/06/81b1d71ba0567ff39d0f98f3637e810846df92f6733aee46004a194b51ea/elasticsearch-8.9.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 255.7 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for urllib3<2,>=1.26.2 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.5/395.5 kB 306.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 195.6 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 599.9 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
from elasticsearch import Elasticsearch
import os
from unidecode import unidecode
import string
import re
#import torch
import numpy as np

es = Elasticsearch("http://localhost:9200")

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('vietdata/vietnamese-content-cls')
tokenizer = AutoTokenizer.from_pretrained('vietdata/vietnamese-content-cls')

In [ ]:
def predict_class(sentence, model, tokenizer):
    clas = ['Agriculture, food, and drink',
 'Albums',
 'Architecture',
 'Art',
 'Biology and medicine',
 'Chemistry and materials science',
 'Classical compositions',
 'Computing and engineering',
 'Earth science',
 'Film',
 'Geography',
 'Language and literature',
 'Mathematics and mathematicians',
 'Media and drama',
 'Other music articles',
 'Philosophy',
 'Physics and astronomy',
 'Places',
 'Religion',
 'Royalty, nobility, and heraldry',
 'Songs',
 'Television',
 'Transport',
 'World history',
 'Armies and military units',
 'Baseball',
 'Basketball',
 'Battles, exercises, and conflicts',
 'Culture, sociology, and psychology',
 'Economics and business',
 'Education',
 'Football',
 'Hockey',
 'Law',
 'Magazines and print journalism',
 'Military aircraft',
 'Military decorations and memorials',
 'Military people',
 'Motorsport',
 'Multi-sport event',
 'Other sports',
 'Politics and government',
 'Pro wrestling',
 'Recreation',
 'Video games',
 'Warships and naval units',
 'Weapons, equipment, and buildings']
    input_ids = torch.tensor([tokenizer.encode(sentence)])
    with torch.no_grad():
            out = model(input_ids)
            probs = out.logits.softmax(dim=-1).tolist()[0]
            top_2_indices = np.argsort(probs)[-2:]
    return clas[top_2_indices[0]], clas[top_2_indices[1]]

In [ ]:
predict_class("Thủ tướng đi ngoại giao", model, tokenizer)

In [10]:
# def preprocess(txt):
#     # Code here
#     ###########
#     return predict_class(txt, model, tokenizer)

def normalize_index(txt: str)->str:
    translator = str.maketrans('', '', string.punctuation)
    # Use the translate method to remove punctuation
    rs = txt.translate(translator)
    rs = unidecode(rs).strip()
    rs = rs.replace(' ', '_')
    return rs.lower()

In [18]:
data_dir = 'C:\\Users\\DaiTran\\Desktop\\University\\Nam-4\\Ky-2\\Do-an-thiet-ke-HTTT\\project\\DAHTTT\\data'

In [19]:
df = None
for file in os.listdir(data_dir):
    f_path = os.path.join(data_dir, file)
    if df == None:
        df = spark.read.option("encoding", "utf-8").json(f_path)
    else:
        new_df = spark.read.option("encoding", "utf-8").json(f_path)
        df = df.union(new_df)

# df.show()
rdd = df.rdd

# processed_rdd = rdd.map(preprocess)

# processed_rdd

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\DaiTran\\Desktop\\University\\Nam-4\\Ky-2\\Do-an-thiet-ke-HTTT\\project\\DAHTTT\\data'

In [8]:
rdd.collect()

23/08/06 21:18:10 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


[Row(comments=0, likes=0, page='Báo Giao thông', post_id='859851818935377', text='Trên xe hơi truyền thống, tùy từng hãng xe thiết lập từ 12 - 64 loại biểu tượng khác nhau, đôi khi gây bối rối cho người lái.\nĐơn giản nhất là cảnh báo “Chưa cài dây an toàn” hầu như xe nào cũng có, cho đếnXem thêm phức tạp như cảnh báo “Hệ thống treo gặp trục trặc”, "Sấy nhiên liệu"...\nGiải mã chính xác những biểu tượng nhỏ xuất hiện trong bảng điều khiển ô tô đang cảnh báo điều gì.\n\nXE.BAOGIAOTHONG.VN\nỨng dụng điện thoại giải mã mọi biểu tượng trên bảng điều khiển ô tô', timestamp='1691220601'),
 Row(comments=12, likes=129, page='𝙋𝙖𝙜𝙚 𝙣𝙖𝙮 𝙘𝙤 𝙣𝙝𝙖𝙘 𝙘𝙝𝙞𝙡𝙡', post_id='824252225941127', text='thời gian chưa bao giờ lên tiếng, nhưng nó lại trả lời hết thắc mắc của chúng ta.', timestamp='1689519936'),
 Row(comments=0, likes=0, page='Kênh Nhạc Hay', post_id='752651226871049', text='Pháp Kiều khi biết mình ở bảng F gồm SMO, 24k. RIGHT, Tọi. 😂', timestamp='1691137855'),
 Row(comments=0, likes=0, page='VTV Mus

In [6]:
index_list = es.cat.indices(format="json")

In [ ]:
# Xóa từng index trong danh sách
for index in index_list:
    index_name = index['index']
    es.indices.delete(index=index_name)

print("All indices have been deleted.")

In [9]:
for x in rdd.collect():
  try:
    #cate1, cate0 = predict_class(x['text'], model, tokenizer)
    data = {   
            'page': x['page'],
            'post_id': x['post_id'],
            'text': x['text'],
            'timestamp': x['timestamp'],
            'likes': int(x['likes']),
            'comments': int(x['comments']),
          }
    #print(cate1, cate0)
    es.index(index=normalize_index(x['page']), document=data)
    #es.index(index=normalize_index(cate0), document=data)

  except:
    continue

23/08/06 21:20:50 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


In [ ]:
for x in rdd.collect():
  try:
    cate1, cate0 = predict_class(x['text'], model, tokenizer)
    data = {   
            'page': x['page'],
            'post_id': x['post_id'],
            'text': x['text'],
            'timestamp': x['timestamp'],
            'likes': int(x['likes']),
            'comments': int(x['comments']),
          }
    print(cate1, cate0)
    es.index(index=normalize_index(cate1), document=data)
    es.index(index=normalize_index(cate0), document=data)

  except:
    continue

In [12]:
from datetime import datetime

timestamp_unix = int('1673094315')  # Chuỗi số timestamp Unix

# Chuyển đổi timestamp Unix sang đối tượng datetime
timestamp_obj = datetime.fromtimestamp(timestamp_unix)

print("Đối tượng ngày tháng:", timestamp_obj)
print("Năm:", timestamp_obj.year)
print("Tháng:", timestamp_obj.month)
print("Ngày:", timestamp_obj.day)

Đối tượng ngày tháng: 2023-01-07 19:25:15
Năm: 2023
Tháng: 1
Ngày: 7


In [1]:
import pandas as pd

In [5]:
# Lấy danh sách các index
indices = es.cat.indices(format="json")

# Khởi tạo danh sách để chứa dữ liệu từ tất cả các index
all_data = []

# Lặp qua danh sách các index và lấy dữ liệu từ mỗi index
for index_info in indices:
    index_name = index_info['index']
    query = {
        "query": {
            "match_all": {}
        },
        "size": 10000  # Số lượng tối đa các tài liệu lấy về cho mỗi index
    }
    response = es.search(index=index_name, body=query)
    hits = response['hits']['hits']
    data = [hit['_source'] for hit in hits]
    all_data.extend(data)

# Tạo DataFrame từ danh sách dữ liệu từ tất cả các index
df = pd.DataFrame(all_data)

# Hiển thị DataFrame

# Đóng kết nối Elasticsearch khi bạn đã hoàn thành công việc


/tmp/ipykernel_15338/4124011651.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=query)


In [6]:
df.head()

,page,post_id,text,timestamp,likes,comments
0,𝙉𝙝𝙖𝙘 𝙣𝙖𝙮 𝙘𝙝𝙞𝙡𝙡 𝙥𝙝𝙚𝙩,304586275482387,"Xin vĩnh biệt cụ Dương Văn Ngộ, người giữ hồn ...",1691197202,405,3
1,𝙉𝙝𝙖𝙘 𝙣𝙖𝙮 𝙘𝙝𝙞𝙡𝙡 𝙥𝙝𝙚𝙩,304253022182379,Dỗi gòi xê ra ik\n-\nSound: ThichThich - Phươn...,1691145790,3759,591
2,𝙉𝙝𝙖𝙘 𝙣𝙖𝙮 𝙘𝙝𝙞𝙡𝙡 𝙥𝙝𝙚𝙩,304370325503982,Khoảng khắc đáng yêu của một gia đình nhảy trê...,1691162186,265,11
3,𝙉𝙝𝙖𝙘 𝙣𝙖𝙮 𝙘𝙝𝙞𝙡𝙡 𝙥𝙝𝙚𝙩,304265158847832,Thêm nhiều hình ảnh tại đám cưới của Thái Vũ t...,1691147650,125,1
4,Kênh Nhạc Sàn,682212903938741,Chỉ cần cậu hạnh phúc thôi. Còn tôi thì sao cũ...,1689595326,0,0
